Just an idea about cleaning data.

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import style
%matplotlib inline 
%matplotlib nbagg

/Users/wannyin/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
ju = pd.read_csv('jun/listings.csv',low_memory=False)
original_length = len(ju)
print original_length

38810


In [3]:
# select the features we want to use
df = ju.loc[:,[u'summary', u'zipcode',
       u'neighborhood_overview', u'transit', u'house_rules', u'host_since', 
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_neighbourhood', u'host_verifications',
       u'host_identity_verified', u'street', u'neighbourhood_group_cleansed',
       u'city',  u'latitude',u'longitude', u'property_type', u'room_type',
       u'accommodates', u'bathrooms', u'bedrooms', u'beds', u'bed_type',
       u'amenities', u'square_feet', u'price', u'weekly_price',
       u'monthly_price', u'security_deposit', u'cleaning_fee',
       u'guests_included', u'extra_people', u'minimum_nights',
       u'maximum_nights', u'availability_30', u'availability_60',
       u'availability_90',u'availability_365',u'number_of_reviews',
       u'last_review', u'review_scores_rating',
       u'review_scores_accuracy', u'review_scores_cleanliness',
       u'review_scores_checkin', u'review_scores_communication',
       u'review_scores_location', u'review_scores_value',u'instant_bookable',
       u'cancellation_policy',u'require_guest_phone_verification',u'reviews_per_month']]

#change price type
df['price'] = df['price'].str.replace('$','')
df['price'] = pd.to_numeric(df['price'],errors = 'coerce')

df['security_deposit'] = df['security_deposit'].str.replace('$','')
df['security_deposit'] = pd.to_numeric(df['security_deposit'],errors = 'coerce')

df['cleaning_fee'] = df['cleaning_fee'].str.replace('$','')
df['cleaning_fee'] = pd.to_numeric(df['cleaning_fee'],errors = 'coerce')

df['extra_people'] = df['extra_people'].str.replace('$','')
df['extra_people'] = pd.to_numeric(df['extra_people'],errors = 'coerce')

df['zipcode'] = pd.to_numeric(df['zipcode'],errors = 'coerce')

#remove rows that have 'NaN' in key features 
remove = df[u'price'].isnull() | df[u'review_scores_rating'].isnull() | df[u'zipcode'].isnull()
df = df[-remove]
remove = df['zipcode']<10000
df = df[-remove]
df = df.reset_index(drop = True)

#fill 'NaN' with '0' in some low priority features
#may consider these features in boolean? or ordinal numbers?
#df = df.fillna(df.mean())

In [4]:
print(df.shape)
#print(df.head())
length = len(df)

(27848, 53)


In [5]:
# set boolean
# add columns as needed

check_summary = df['summary'].isnull()
check_transit = df['transit'].isnull()
check_neighborhood = df['neighborhood_overview'].isnull()

df['summary_bool']=1
df['neigh_overview_bool']=1
df['traffic']=1
df['superhost']=1
df['host_vertified']=1
df['guest_vertified']=1
df['instant_book']=1

for i in range(len(df)):
    if check_summary[i] :
        df.loc[i,'summary_bool']=0
    
    if check_transit[i]:
        df.loc[i,'traffic']=0
        
    if check_neighborhood[i]:
        df.loc[i,'neigh_overview_bool']=0
    
    if df['host_is_superhost'][i] == 'f':
        df.loc[i,'superhost']=0
    
    if df['host_identity_verified'][i] == 'f':
        df.loc[i,'host_vertified']=0
    
    if df['require_guest_phone_verification'][i] == 'f':
        df.loc[i,'guest_vertified']=0
        
    if df['instant_bookable'][i] == 'f':
        df.loc[i,'instant_book']=0       

In [6]:
#set boolean for housing rules
#smoking, pet, party, guest
#add features as needed
rule = df['house_rules'].str.lower()

smoking = rule.str.contains("smoke|smoking", na=False)
df.loc[:,'smoking']=-smoking

pet = rule.str.contains("pet", na=False)
df.loc[:,'pet']=-pet

party = rule.str.contains("party|parties", na=False)
df.loc[:,'party']=-party

guest = rule.str.contains("guest|guests", na=False)
df.loc[:,'guest']=-guest

In [7]:
#change host_since to duration
date_scrap = "2016-07-01"
date_scrap = pd.to_datetime(date_scrap).date()

check_host_since = df['host_since'].isnull()
check_last_review = df['last_review'].isnull()

duration = [None] * len(df)
last_review = [None] * len(df)

for i in range(len(df)):
    if check_host_since[i] == False:
        duration[i] = (date_scrap-pd.to_datetime(df['host_since'][i]).date()).days 
    if check_last_review[i] == False:
        last_review[i] = (date_scrap-pd.to_datetime(df['last_review'][i]).date()).days

df.loc[:,'duration']=duration
df.loc[:,'last_review_time']=last_review

In [8]:
#change string into percentage
response = df['host_response_rate']
response = pd.to_numeric(response,errors = 'ignore')
check_response = response.isnull()

for i in range(len(response)):
    if check_response[i] == False: 
        response[i] =float(response[i].strip('%'))/100
        
df['host_response'] = response

acceptance = df['host_acceptance_rate']
acceptance = pd.to_numeric(acceptance,errors = 'ignore')
check_acceptance = acceptance.isnull()

for i in range(len(acceptance)):
    if check_acceptance[i] == False: 
        acceptance[i] =float(acceptance[i].strip('%'))/100

df['host_acceptance'] = acceptance

df['availability'] = df['availability_365']/365

In [9]:
#count the methods of host vertification
a = [0 for i in range(length)]
mm = df['host_verifications']

for i in range(length):
    method = len(mm[i].split(','))
    a[i]=method

df['verification_method'] = a

In [10]:
#define room_type as boolean

set(df["room_type"])

df['Entire home/apt']=0
df['Private room']=0
df['Shared room']=0
i=0;

for item in df["room_type"]:
    if item in ['Entire home/apt','Private room','Shared room']:
        df.loc[i,item]=1
    i+=1

In [11]:
#define cancellation_policy as boolean    
set(df["cancellation_policy"])
df['flexible']=0
df['long_term']=0
df['moderate']=0
df['no_refunds']=0
df['strict']=0
df['super_strict_30']=0
df['super_strict_60']=0
i=0;

for item in df["cancellation_policy"]:
    if item in ['flexible','moderate','no_refunds','strict','super_strict_30','super_strict_60','long_term']:
        df.loc[i,item]=1
    i+=1

In [12]:
#define host_response_time as boolean    
set(df["host_response_time"])

df['within an hour']=0
df['within a few hours']=0
df['within a day']=0
df['a few days or more']=0

i=0;

for item in df["host_response_time"]:
        df.loc[i,item]=1
    i+=1

In [13]:
#define bed_type as boolean 
set(df["bed_type"])

df['Airbed']=0
df['Couch']=0
df['Futon']=0
df['Pull-out Sofa']=0
df['Real Bed']=0

i=0;

for item in df["bed_type"]:
    if item in ['Airbed','Couch','Futon','Pull-out Sofa','Real Bed']:
        df.loc[i,item]=1
    i+=1


amenity

In [14]:
#create a list containing all amenities from each host
amenities = list(df['amenities'])
#merge all amenities from hosts into one single list
am = ','.join(amenities)
#remove the left and right brackets
am = am.replace('{','')
am = am.replace('}','')
#split each element with a comma
am = am.split(',')

In [15]:
#Turn the list of amenities into a set
amenity_items = list(set(am))
amenity_items = list(filter(None, amenity_items))

#Remove quotations of each item
for i in range(len(amenity_items)):
    amenity_items[i] = re.sub(r'^"|"$', '', amenity_items[i])

#Remove wrong data values called "Translation missing"  
for item in amenity_items:
    match = re.match(r'translation',item)
    if match:
        amenity_items.remove(item)

In [16]:
for i in range(len(amenities)):
    amenities[i] = amenities[i].replace('{','')
    amenities[i] = amenities[i].replace('}','')
    amenities[i] = amenities[i].split(',')
    for j in range(len(amenities[i])):
        amenities[i][j] = re.sub(r'^"|"$', '', amenities[i][j])

In [17]:
for item in amenity_items:
    match = re.match(r'Washer',item)
    if match in amenities[1]:
        print(item)

In [18]:
columns = amenity_items
new_table = pd.DataFrame(index = df.reset_index().values[:,0],columns = columns)
new_table = new_table.fillna(0)
#new_table.head()

In [19]:
for i in range(len(amenities)):
    for item in amenity_items:
        if ( item in amenities[i] ):
            new_table.set_value(i, item, 1)

#new_table.head()

In [20]:
df = pd.concat([df, new_table], axis=1)

In [21]:
zillow_data=pd.read_csv('Zip_Zri_AllHomesPlusMultifamily_Summary.csv',low_memory=False)
df_zillow=pd.concat([zillow_data['RegionName'], zillow_data['Zri']],axis=1)
df_zillow.columns=['zipcode','Zillow_Rent_Index']
#merge zillow rental index into listing data set
df=pd.merge(df, df_zillow, on='zipcode')

In [34]:
new_df = pd.DataFrame()

s = [u'price','host_response',u'accommodates', u'traffic', u'superhost',
       u'bathrooms', u'bedrooms', u'beds',u'summary_bool', u'neigh_overview_bool',
       u'host_vertified', u'guest_vertified', u'instant_book', 
       u'pet', u'party', u'guest', u'duration', u'last_review_time',
       u'host_acceptance', u'availability',
       u'verification_method', u'review_scores_rating', u'review_scores_accuracy',
       u'review_scores_cleanliness', u'review_scores_checkin',
       u'review_scores_communication', u'review_scores_location',
       u'review_scores_value',u'security_deposit', u'cleaning_fee', u'guests_included',
       u'extra_people', u'minimum_nights', u'maximum_nights',u'location_price',
       u'Other pet(s)', u'Essentials', u'Carbon Monoxide Detector',
       u'Elevator in Building', u'Indoor Fireplace', u'Internet', u'Washer',
       u'Hangers', u'Buzzer/Wireless Intercom', u'Washer / Dryer', u'TV',
       u'Gym', u'Fire Extinguisher', u'Hot Tub', u'Cable TV',
       u'Air Conditioning', u'Suitable for Events', u'Kitchen',
       u'Family/Kid Friendly', u'Shampoo', u'Heating', u'Hair Dryer',
       u'Smoke Detector', u'Iron', u'Lock on Bedroom Door',
       u'Free Parking on Premises', u'Pets live on this property',
       u'Safety Card', u'Doorman', u'Dryer', u'Pool', u'Pets Allowed',
       u'Wheelchair Accessible', u'First Aid Kit', u'Dog(s)',
       u'Wireless Internet', u'Laptop Friendly Workspace', u'Breakfast',
       u'Smoking Allowed', u'Cat(s)', u'24-Hour Check-in','Entire home/apt',
       'Private room','Shared room','flexible','moderate',
       'no_refunds','strict', 'super_strict_30', 'super_strict_60','within an hour',
       'within a few hours', 'within a day', 'a few days or more', 'Airbed',
       'Couch', 'Futon', 'Pull-out Sofa', 'Real Bed','Zillow_Rent_Index']

new_df = df.loc[:,s]

In [35]:
s = [u'duration', u'host_response', u'host_acceptance',
u'review_scores_accuracy', u'review_scores_cleanliness',u'review_scores_checkin', 
u'review_scores_communication',u'review_scores_location', u'review_scores_value']

In [36]:
new_df[s] = new_df[s].fillna(new_df[s].mean())
new_df = new_df.fillna(0)

In [37]:
new_df.to_csv('where_df.csv')

In [38]:
new_df

,price,host_response,accommodates,traffic,superhost,bathrooms,bedrooms,beds,summary_bool,neigh_overview_bool,...,within an hour,within a few hours,within a day,a few days or more,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,Zillow_Rent_Index
0,65.0,1.000000,2,1,0,1.0,1.0,1.0,1,0,...,1,0,0,0,0,0,0,0,1,1904
1,125.0,1.000000,3,1,0,1.0,1.0,1.0,1,1,...,0,1,0,0,0,0,0,0,1,1904
2,79.0,1.000000,4,1,0,1.0,1.0,2.0,1,1,...,1,0,0,0,0,0,0,0,1,1904
3,59.0,1.000000,4,1,1,1.0,1.0,1.0,1,1,...,1,0,0,0,0,0,0,0,1,1904
4,99.0,1.000000,4,1,1,1.0,1.0,2.0,0,1,...,0,1,0,0,0,0,0,0,1,1904
5,179.0,1.000000,5,1,0,2.5,2.0,2.0,1,1,...,1,0,0,0,0,0,0,0,1,1904
6,95.0,1.000000,2,1,0,1.0,0.0,1.0,1,1,...,0,1,0,0,0,0,0,0,1,1904
7,85.0,1.000000,2,1,0,1.0,1.0,1.0,1,1,...,1,0,0,0,0,0,0,0,1,1904
8,49.0,1.000000,3,0,0,1.0,1.0,2.0,1,0,...,1,0,0,0,0,0,0,0,1,1744
9,75.0,0.900000,2,1,0,1.0,0.0,1.0,1,0,...,0,0,1,0,0,0,0,0,1,1744


ignore the codes below

In [27]:
#try to seperate data in different zones

set(df["neighbourhood_group_cleansed"])

data_Brooklyn=df[df["neighbourhood_group_cleansed"]=='Brooklyn']
data_Manhattan=df[df["neighbourhood_group_cleansed"]=='Manhattan']
data_Queens=df[df["neighbourhood_group_cleansed"]=='Queens']
data_Bronx=df[df["neighbourhood_group_cleansed"]=='Bronx']
data_StatenIsland=df[df["neighbourhood_group_cleansed"]=='Staten Island']